<a href="https://colab.research.google.com/github/nghiho112/color-accessibility-app/blob/model/Color_Classifier_DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pathlib
import matplotlib.pyplot as plt

In [2]:
# Importing Tensorflow
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.15.0


In [4]:
#install this one before put everything in .py file
!pip install git+https://github.com/tensorflow/docs # Use some functions from tensorflow_docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-vth71ztx
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-vth71ztx
  Resolved https://github.com/tensorflow/docs to commit b64768499123da8b2253a534277d62e20de3ec73
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-docs: filename=tensorflow_docs-2024.2.5.73858-py3-none-any.whl size=182442 sha256=3c5c7d4ffc0cdbe5c9770ebeb6b11ff0e301e69cb06e3b249c3960bcdf188cf4
  Stored in directory: /tmp/pip-ephem-wheel-cache-9esnyylu/wheels/86/0f/1e/3b62293c8ffd0fd5a49508e6871cdb7554abe9c62afd35ec53
Successfully built tensorflow-docs


In [5]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
#might not need it (will be deleted)

#plotly used for plotting Interactive Graphs
!pip install plotly
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go

In [6]:
#Loading data set
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import os

Mounted at /content/drive


In [7]:
data_path = '/content/drive/MyDrive/Color/final_data.csv'

if not os.path.exists(data_path):
  print("The path doesn't exist!")

dataset = pd.read_csv(data_path)
dataset

,red,green,blue,label
0,20,139,240,Blue
1,174,83,72,Brown
2,144,249,131,Green
3,168,25,156,Pink
4,30,182,136,Green
...,...,...,...,...
5047,26,26,26,Black
5048,27,27,27,Black
5049,28,28,28,Black
5050,29,29,29,Black


In [8]:
#preprocessing dataset

#To find whether null entries are present
dataset.isnull().sum()


#One-hot-encoding to convert categirical data int numerric data
dataset.label.unique()
dataset = pd.get_dummies(dataset, columns=['label'])
#dataset


#reorder
dataset.columns
dataset = dataset[['red', 'green', 'blue', 'label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']]
print(dataset)

#split data into training and testing set
train_dataset = dataset.sample(frac=0.8, random_state=8) #train = 80%,  random_state = any int value means every time when you run your program you will get the same output for train and test dataset, random_state is None by default which means every time when you run your program you will get different output because of splitting between train and test varies within
test_dataset = dataset.drop(train_dataset.index) #remove train_dataset from dataframe to get test_dataset
print(train_dataset)

#split features: red-green-blue and labels
train_labels = pd.DataFrame([train_dataset.pop(x) for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']]).T
#print(train_label)

test_labels = pd.DataFrame([test_dataset.pop(x) for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']]).T




      red  green  blue  label_Red  label_Green  label_Blue  label_Yellow  \
0      20    139   240          0            0           1             0   
1     174     83    72          0            0           0             0   
2     144    249   131          0            1           0             0   
3     168     25   156          0            0           0             0   
4      30    182   136          0            1           0             0   
...   ...    ...   ...        ...          ...         ...           ...   
5047   26     26    26          0            0           0             0   
5048   27     27    27          0            0           0             0   
5049   28     28    28          0            0           0             0   
5050   29     29    29          0            0           0             0   
5051   30     30    30          0            0           0             0   

      label_Orange  label_Pink  label_Purple  label_Brown  label_Grey  \
0             

#Try decision tree + Random forest

In [9]:
#normalize data
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = StandardScaler()

train_dataset_x = scaler.fit_transform(train_dataset)
train_dataset_x
test_dataset_x = scaler.transform(test_dataset)



In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Initialize the model
decision_tree = DecisionTreeClassifier(random_state=8)

# Fit the model on the training data
decision_tree.fit(train_dataset, train_labels)

# Predict on the test data
dt_predictions = decision_tree.predict(test_dataset)

# Calculate the accuracy
dt_accuracy = accuracy_score(test_labels, dt_predictions)
print("Decision Tree Accuracy:", dt_accuracy)


Decision Tree Accuracy: 0.8118811881188119


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping, ModelCheckpoint

#parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt'],

}


# Initialize the model
random_forest = RandomForestClassifier(random_state=8)

# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

grid_search = GridSearchCV(estimator=random_forest, param_grid= param_grid, cv =3, n_jobs =-1, verbose =0, scoring = 'accuracy' )


# Fit the model on the training data
grid_search.fit(train_dataset_x, train_labels)

best_param = grid_search.best_estimator_
# Predict on the test data
rf_predictions = best_param.predict(test_dataset_x)

# Calculate the accuracy
rf_accuracy = accuracy_score(test_labels, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)


KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Initialize the model
random_forest = RandomForestClassifier(random_state=8)

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Create the GridSearchCV object with early stopping
grid_search = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv=3, n_jobs=-1, verbose=0, scoring='accuracy', callbacks=[early_stopping])

# Fit the model on the training data
grid_search.fit(train_dataset_x, train_labels)

# Predict on the test data
rf_predictions = grid_search.best_estimator_.predict(test_dataset_x)

# Calculate the accuracy
rf_accuracy = accuracy_score(test_labels, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

TypeError: GridSearchCV.__init__() got an unexpected keyword argument 'callbacks'

#Save model

In [ ]:
#save model
#Install PyDrive library into Google Colab notebook
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

!pip install pyyaml h5py  # Required to save models in HDF5 format

# Save Model or weights on google drive and create on Colab directory in Google Drive

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('colormodel.keras')
model_file = drive.CreateFile({'title' : 'colormodel.keras'}) # colormodel_acu_  .h5 is the name of the model uploaded to Drive
model_file.SetContentFile('colormodel.keras')
model_file.Upload()



In [ ]:
!pip install joblib
import joblib

# Assuming 'model' is the variable containing your trained model
# Replace 'your_model.pkl' with the desired file name
joblib.dump(model, 'color.pkl')




['color.pkl']

In [ ]:
from google.colab import files

files.download('color.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# download to google drive
drive.CreateFile({'id': model_file.get('id')}) ### very important to run this last cell for loading the model in new notebook

GoogleDriveFile({'id': '1lNW-RjpGB0VAWjjMqAwK3H_IzVenJd4c'})

In [ ]:
#Load model from google drive into colab
file_obj = drive.CreateFile({'id': '1lNW-RjpGB0VAWjjMqAwK3H_IzVenJd4c'})
file_obj.GetContentFile('colormodel.keras') #colormodel_trained.h5 is the name of model uploaded to colab

In [ ]:
#will be remove in .py
ls #check for the saved model on googledrive

NameError: ignored

#Img testing cell

In [ ]:
import numpy as np
from PIL import Image

#load model to run prediction
# Recreate the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model('colormodel.keras') #very important

# Load the image using PIL (Pillow)
image_path = '/content/drive/MyDrive/Color/test_set/Red_Rose.JPG'
image = Image.open(image_path)

# new_size = (10,10)
# image = image.resize(new_size)

plt.imshow(image) #remove in .py
plt.show()        #remove in .py

# Convert the image to a NumPy array
image_array = np.array(image)

# Extract the red, green, and blue channels into separate arrays
red_channel = image_array[:, :, 0]
green_channel = image_array[:, :, 1]
blue_channel = image_array[:, :, 2]

# Create a DataFrame with the red, green, and blue channels
import pandas as pd

image_df = pd.DataFrame({'red': red_channel.ravel(),
                         'green': green_channel.ravel(),
                         'blue': blue_channel.ravel()})

# Display the DataFrame
print(image_df)

#Predict the data:
predictions = model.predict(image_df)

#selecting class with highest confidence
actual_encoded_train_labels = np.argmax(train_labels.to_numpy(), axis=1) #train_labels were originally in one-hot

predicted_encoded_train_labels = np.argmax(predictions, axis=1)

predicted_encoded_train_labels = pd.DataFrame(predicted_encoded_train_labels, columns=['Labels'])
predicted_encoded_train_labels

predicted_encoded_train_labels.to_csv('predicted_labels.csv', index =False)


NameError: ignored